# Práctica 6
## Propuesta: Intento de sistema de reconocimiento facial y reacciones según datos de DeepFace
<ul>
<li>Nikhil Chandru Durgadas Chellaram</li>
<li>Raúl Mateus Sánchez</li>
</ul>

Para comprobar el desarrollo de la práctica: https://github.com/raulmat19/Vision-por-Computador/tree/main/Practica-6

In [21]:
import cv2
from deepface import DeepFace
import matplotlib.pyplot as plt
import os
import numpy as np

In [22]:
models = ["VGG-Face","Facenet","Facenet512","OpenFace","DeepFace","DeepID","ArcFace","Dlib","SFace"]

metrics = ["cosine", "euclidean", "euclidean_l2"]

backends = ['opencv','ssd','dlib','mtcnn','retinaface','mediapipe']

Se ha implementado una pequeña función con nombre <strong>draw_text()</strong> que tiene como objetivo representar texto en el contenido capturado por la webcam a tiempo real. Se usará esta función mas adelante en el notebook

In [23]:
def draw_text(img, text,
          font=cv2.FONT_HERSHEY_SIMPLEX,
          pos=(0, 0),
          font_scale=3,
          font_thickness=2,
          text_color=(255, 255, 255),
          text_color_bg=(0, 0, 0)
          ):

    x, y = pos
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_w, text_h = text_size
    cv2.rectangle(img, pos, (x + text_w + 5, y + text_h + 5), text_color_bg, -1)
    cv2.putText(img, text, (x, int(y + text_h + font_scale - 1)), font, font_scale, text_color, font_thickness)

    return text_size

Esta función se encarga de realizar una serie de fotos al usuario para poder introducirlo en la "base de datos" de usuarios registrados. El objetivo de esta función es básicamente incluir al usuario en cuestión en el conjunto de usuarios reconocidos para que pueda disfrutar de toda la funcionalidad propuesta.

Como la cantidad de fotos que se van a tomar son 3, se recomienda al usuario usar 3 distancias, la primera bastante cercana a la cámara, de manera que salga únicamente la cara, la segunda foto sería una distancia media, es decir, que se pueda ver hasta los hombros del usuario, y una tercera ya puede ser una foto un poco más lejana aún.

Cabe destacar que el método de tomar fotos es con la utilización de la tecla <strong>ESPACIO</strong>

In [24]:
def getFace(path, name):
    vid = cv2.VideoCapture(0)
    counter = 0

    while(True):      
        # fotograma a fotograma
        ret, frame = vid.read()

        if ret:
            draw_text(frame, f"{counter}", font_scale=1, pos=(50, 10), text_color_bg=(0, 0, 0))
            cv2.imshow('Cam', frame)   

        key = cv2.waitKey(5)     
        # Detenemos pulsado ESC
        if key and key == 27 or counter == 3:
            break
        elif key and key == 32:
            if counter > 0:
                new_path = path + '/' + name + str(counter) + '.jpg'
            
            else:
                new_path = path + '/' + name + '.jpg'

            cv2.imwrite(new_path, frame)
            counter += 1
            
            
    # Libera el objeto de captura
    vid.release()
    # Destruye ventanas
    cv2.destroyAllWindows()

Con este pequeño sistema, se solicita introducir una serie de características diferenciales del propio usuario para poder registrar su cara en el sistema con la finalidad de simular un sistema de reconocimiento facial así como prodecer a las reacciones propuestas empleando las utilidades de DeepFace

In [25]:
face_loaded = False

data_loaded = input ("Si su cara ha sido ya registrada, introduzca 'Si'. En caso contrario, introduzca 'No' o cualquier cosa")
if data_loaded == 'Si' or data_loaded == 'si':
    face_loaded = True

if not face_loaded:
    data = False
    escape = 0

    while not data:

        name = input ("Introduce tu nombre: ")
        surname = input ("Introduce tus apellidos: ")
        age = input ("Introduce tu edad: ")
        sexo = input ("Introduce tu sexo: ")
        workingStatus = input ("Introduce tu ocupación: ")

        if name == '' or surname == '' or age == '' or sexo == '' or workingStatus == '':
            warning = input ("Datos incorrectos. Pulse Enter para volver a introducirlos")
            escape += 1
        
        else:
            data = True

        if escape >= 2:
            break

    if data:
        try:
            for file in os.listdir("face_recognition_images/"):
                if file.startswith("representations"):
                    os.remove("face_recognition_images/" + file)
            
            dir_path = 'face_recognition_images/' + str(name)
            try:
                os.stat(dir_path)
            except:
                os.mkdir(dir_path)
            file_path = dir_path + '/info.txt'
            with open(file_path, 'w') as f:
                f.write(f"{name}, {surname}, {age}, {sexo}, {workingStatus}")
            f.close()

            warning = input ("Pulse Enter para escanear su cara. Debe sacarse tres fotos, empleando para ello la tecla 'Barra espaciadora'")
            getFace(dir_path, name)

        except:
            print("Fallo del sistema")

https://stackoverflow.com/questions/60674501/how-to-make-black-background-in-cv2-puttext-with-python-opencv

Para darle algo de utilidad diferenciadora al reconocimiento de raza, se ha decidido darle otro punto de vista a esta función, en este preciso caso, se ha decidido aprovechando las épocas deportivas del Mundial de fútbol de Catar 2022, pues según la raza que se obtenga se mostrará la Selección de Fútbol que apoya cada usuario. 

Para ello se han creado varias carpetas con cada tipo de posible raza, asociándole una serie de banderas de países a cada una de ellas. Por ejemplo, si sale que el usuario es de raza "middle eastern" o de Oriente Medio, entonces se le han asignado las posibles banderas de Arabia, Catar, Irán o Marruecos. Y el sistema de manera totalmente aleatoria le asignará una de esas banderas a representar en el mundial.

Las siguientes dos funciones se encargan de navegar por las distintas carpetas según el tipo de raza y mostrar en tiempo real la selección que apoya cada usuario en cuestión.

In [26]:
def showFlag(ruta, img):
    img_flag = cv2.imread(ruta)
    img_flag = cv2.resize(img_flag, (150, 100))
    draw_text(img, "En el Mundial apoyas a: ", font_scale=1, pos=(50, img.shape[0]-50), text_color_bg=(0, 0, 0))
    img[img.shape[0]-100:img.shape[0], img.shape[1]-150:img.shape[1]] = img_flag

In [27]:
def getFlag(raza):
    if raza == "white":
        return "flags_images/white"
    elif raza == "asian":
        return "flags_images/asian"
    elif raza == "indian":
        return "flags_images/indian"
    elif raza == "middle eastern":
        return "flags_images/middle-eastern"
    elif raza == "latino hispanic":
        return "flags_images/latino-hispanic"
    elif raza == "black":
        return "flags_images/black"

En esta última parte se trata el sistema de reconocimiento facial. Lo que se encarga de realizar esta sección es analizar y comparar en tiempo real el usuario que aparece en la webcam con las caras de los usuarios ya registrados previamente. 

Si el usuario no ha sido registrado previamente, el sistema no lo reconocerá y le saldrá un mensaje de texto que pondrá <strong>Bloqueado</strong>

En caso contrario, el sistema lo reconocerá y le saldrá un mensaje de texto que pondrá <strong>Desbloqueado</strong>, y posteriormente se mostrará toda la información que el usuario introdujo a la hora de registrarse en el sistema.

Una vez que se ha mostrado toda la información del usuario por pantalla, se ha incluido la opción de mostrar el país al que apoya en el Catar 2022. Para acceder a este modo el usuario debe pulsar la tecla <strong>D</strong> en el teclado. 

Cabe destacar que si el sistema no ha reconocido al usuario en cuestión, la funcionalidad de la tecla <strong>D</strong> queda reducida a ninguna, puesto que le saldría un mensaje que pondría: <br> <i>Accion bloqueada hasta identificacion. Pulse d para ello</i>

In [28]:
#%%capture
vid = cv2.VideoCapture(0)
counter = 0
init = False
flag_mode = False
line = None
y0, dy = 10, 4
properties = ["Nombre", "Apellidos", "Edad", "Sexo", "Ocupacion"]
prev_race_path = None
prev_race = None

while(True):      
    # fotograma a fotograma
    ret, frame = vid.read()

    if ret:
        # Sistema de reconocimiento facial y muestra de información de usuario
        if not flag_mode:
            if counter == 100 or not init:
                cv2.imwrite('cap_faces/face.jpg', frame)

                try:
                    df = DeepFace.find(img_path = 'cap_faces/face.jpg', db_path = "face_recognition_images/", model_name=models[2], distance_metric = metrics[2], prog_bar = False, enforce_detection=False, silent = True)
                    path = df.at[0, 'identity']
                    distance = df.at[0, 'Facenet512_euclidean_l2']
                    if float(distance) <= 0.75:
                        info_path = os.path.dirname(path) + "/info.txt"
                        with open(info_path) as f:
                            line = f.readline()
                        f.close()

                        draw_text(frame, "Desbloqueado", font_scale=1, pos=(0, frame.shape[0]-50), text_color_bg=(0, 0, 0))
                        for i, line in enumerate(line.split(", ")):
                            y = y0 + i*dy*7
                            draw_text(frame, f"{properties[i]}:  {line}", font_scale=1, pos=(5, y), text_color_bg=(0, 0, 0))
                        
                        counter == 0
                        init = True
                    else:
                        init = False
                        print("Not recognised")
                        draw_text(frame, "Bloqueado", font_scale=1, pos=(0, frame.shape[0]-50), text_color_bg=(0, 0, 0))

                except:
                    init = False
                    print("Not recognised")
                    draw_text(frame, "Bloqueado", font_scale=1, pos=(0, frame.shape[0]-50), text_color_bg=(0, 0, 0))

            elif init and counter != 100:
                
                draw_text(frame, "Desbloqueado", font_scale=1, pos=(0, frame.shape[0]-50), text_color_bg=(0, 0, 0))
                for i, line in enumerate(line.split(", ")):
                    y = y0 + i*dy*7
                    draw_text(frame, f"{properties[i]}:  {line}", font_scale=1, pos=(5, y), text_color_bg=(0, 0, 0))
                
                counter += 1

        elif flag_mode and init:
            obj = DeepFace.analyze(img_path = frame, actions = ['race'], enforce_detection=False)
            race = obj['dominant_race']
            if race != prev_race:
                path_to_flag = getFlag(race)
                flag_list = os.listdir(path_to_flag)
                index = np.random.randint(0, len(flag_list))
                path_to_flag = path_to_flag + "/" + flag_list[index]
                showFlag(path_to_flag, frame)
                prev_race_path = path_to_flag
                prev_race = race

            else:
                showFlag(prev_race_path, frame)

            draw_text(frame, f"Raza: {race}", font_scale=1, pos=(0, frame.shape[0]-470), text_color_bg=(0, 0, 0))

        elif flag_mode and not init:
            draw_text(frame, "Accion bloqueada hasta identificacion.", font_scale=1, pos=(0, frame.shape[0]-80), text_color_bg=(0, 0, 0))
            draw_text(frame, "Pulse d para ello", font_scale=1, pos=(0, frame.shape[0]-50), text_color_bg=(0, 0, 0))
        cv2.imshow('Cam', frame)
                    
    # Detenemos pulsado ESC
    key = cv2.waitKey(5)
    if key == 27:
        break
    if key == ord('d'):
        print("pulsando la d")
        if flag_mode:
            flag_mode = False
            init = False
        else:
            flag_mode = True
  
# Libera el objeto de captura
vid.release()
# Destruye ventanas
cv2.destroyAllWindows()

Finding representations:   0%|          | 0/6 [00:00<?, ?it/s]

1/1 [==============================] - 0s 83ms/step


Finding representations:  17%|█▋        | 1/6 [00:00<00:01,  4.58it/s]

1/1 [==============================] - 0s 75ms/step


Finding representations:  33%|███▎      | 2/6 [00:00<00:00,  4.80it/s]

...

1/1 [==============================] - 0s 77ms/step


Analyzing:   0%|          | 0/1 [00:00<?, ?it/s]

1/1 [==============================] - 0s 76ms/step


Analyzing: 100%|██████████| 1/1 [00:00<00:00,  4.10it/s]


pulsando la d
1/1 [==============================] - 0s 269ms/step


En desarrollo ...